In [1]:
import cv2
import os
import time
import torch
import numpy as np
from torch import nn
from torchvision import models

In [2]:
models.vgg19(pretrained=True).features

c:\Users\k26ra\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\k26ra\AppData\Local\Programs\Python\Python310\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


Sequential(
  (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace=True)
  (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace=True)
  (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace=True)
  (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace=True)
  (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace=True)
  (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace=True)
  (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace=True)
  (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (17): ReLU(inplace=True)
  (18): MaxPoo

In [3]:
def capture(filename, timesep, rgb, h, w):
  frames = np.zeros((timesep, rgb, h, w), dtype=float)
  i = 0
  vc = cv2.VideoCapture(filename)
  if vc.isOpened():
    rval, frame = vc.read()
    while rval and i < timesep:
      # Resize the frame to the specified width and height
      frm = cv2.resize(frame, (w, h))
      frm = np.expand_dims(frm, axis=0)
      frm = np.moveaxis(frm, -1, 1)
      if np.max(frm) > 1:
        frm = frm / 255.0
      frames[i][:] = frm
      i += 1
      rval, frame = vc.read()
    vc.release()  # Release the video capture object
  else:
    print("Error: Unable to open video file")
  return frames

In [4]:
class TimeWarp(nn.Module):
  '''video to frames'''

  def __init__(self, baseModel, method='sqeeze'):
    super(TimeWarp, self).__init__()
    self.baseModel = baseModel
    self.method = method

  def forward(self, x):
    batch_size, time_steps, C, H, W = x.size()
    if self.method == 'loop':
      output = []
      for i in range(time_steps):
        # input one frame at a time into the basemodel
        x_t = self.baseModel(x[:, i, :, :, :])
        # Flatten the output
        x_t = x_t.view(x_t.size(0), -1)
        output.append(x_t)
      # end loop
      # make output as  ( samples, timesteps, output_size)
      x = torch.stack(output, dim=0).transpose_(0, 1)
      output = None  # clear var to reduce data  in memory
      x_t = None  # clear var to reduce data  in memory
    else:
      # reshape input  to be (batch_size * timesteps, input_size)
      x = x.contiguous().view(batch_size * time_steps, C, H, W)
      x = self.baseModel(x)
      x = x.view(x.size(0), -1)
      # make output as  ( samples, timesteps, output_size)
      x = x.contiguous().view(batch_size, time_steps, x.size(-1))
    return x

In [5]:
class extractlastcell(nn.Module):
  def forward(self, x):
    out, _ = x
    return out[:, -1, :]

In [6]:
def Model(weight='./Statemamonmixed96accviolance.pth'):
  # Create model
  num_classes = 1
  dr_rate = 0.2
  pretrained = True
  rnn_hidden_size = 80
  rnn_num_layers = 1
  baseModel = models.vgg19(pretrained=pretrained).features

  i = 0
  for child in baseModel.children():
    if i < 28:
      for param in child.parameters():
        param.requires_grad = False
    else:
      for param in child.parameters():
        param.requires_grad = True
    i += 1

  num_features = 12800
  # Example of using Sequential
  model = nn.Sequential(
      TimeWarp(baseModel, method='loop'),
      nn.LSTM(num_features, rnn_hidden_size, rnn_num_layers,
              batch_first=True, bidirectional=True),
      extractlastcell(),
      nn.Linear(160, 256),
      nn.ReLU(),
      nn.Dropout(dr_rate),
      nn.Linear(256, num_classes)
  )
  checkpoint = torch.load(weight, map_location=torch.device('cpu'))
  model.load_state_dict(checkpoint['state_dict'])
  return model

In [7]:
def pred_fight(model, video, acuracy=0.58):
  model.eval()
  outputs = model(video)
  torch.sigmoid(outputs)
  preds = torch.sigmoid(outputs).to('cpu')
  preds = preds.detach().numpy()
  if preds[0][0] >= acuracy:
    return True, preds[0][0]
  else:
    return False, preds[0][0]

In [8]:
np.random.seed(1234)
device = torch.device("cpu")   # Use CPU only
model22 = Model()
model22.to(device)

Sequential(
  (0): TimeWarp(
    (baseModel): Sequential(
      (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (1): ReLU(inplace=True)
      (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (3): ReLU(inplace=True)
      (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (6): ReLU(inplace=True)
      (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (8): ReLU(inplace=True)
      (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (11): ReLU(inplace=True)
      (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (13): ReLU(inplace=True)
      (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
      (15): ReLU(inplace=True)
      (

In [9]:
cap = cv2.VideoCapture(
    './Meanwhile In New York City (Caught On Ring Camera) #shorts.mp4')

cap.isOpened()

# _1st_frame = cap.read()[1]
# _1st_frame.shape

count = 0
while cap.read()[0]:
  count += 1

count

True

951

In [10]:
vid = None


def main_fight(video_path, accuracyfight=0.65):
  global vid

  tmp_obj = {}

  if os.path.exists('./tmp.mp4'):
    os.remove('./tmp.mp4')

  # Open the local video file
  cap = cv2.VideoCapture(video_path)

  # display(cap, dir(cap))

  # Check if the video file was opened successfully
  if not cap.isOpened():
    print("Error: Unable to open video file")
    return None

  else:
    # Open a file to write the video data
    with open("tmp.mp4", "wb") as file:
      while True:
        # Read a frame from the video
        ret, frame = cap.read()
        # Check if the frame was read successfully
        if not ret:
          break  # Break the loop if no more frames are available
        # Write the frame data to the file
        file.write(frame)

  vid = capture("tmp.mp4", 40, 3, 170, 170)

  # display(vid)

  # return

  v = np.array([vid])

  v = torch.from_numpy(v)
  v = v.to(device, dtype=torch.float)
  millis = int(round(time.time() * 1000))
  f, precent = pred_fight(model22, v, acuracy=accuracyfight)
  tmp_obj = {'fight': f, 'precentegeoffight': str(precent)}
  millis2 = int(round(time.time() * 1000))
  tmp_obj['processing_time'] = str((millis2-millis)/1000)
  return tmp_obj

In [12]:
# vid[0]
# vid[39]
#
# vid[0][0]
# vid[0][1]
# vid[0][2]

# vid[0][0][169][169]
# vid[0][1][169][169]
# vid[0][2][169][169]

# vid[1][0][169][169]
# vid[1][1][169][169]
# vid[1][2][169][169]

In [13]:
# np.sum(vid.flatten() == 0)

AttributeError: 'NoneType' object has no attribute 'flatten'

In [ ]:
# type(vid)

numpy.ndarray

In [ ]:
# display(vid.shape)
# v.shape

# v = np.array([vid])
# v = torch.from_numpy(v)
# v[0][0][0].shape
# v[0][0][1].shape
# v[0][0][2].shape

# v[0][0][2].shape

# v[0].shape

torch.Size([170, 170])

torch.Size([170, 170])

torch.Size([170, 170])

torch.Size([40, 3, 170, 170])

In [14]:
# main_fight(
#     video_path="./Meanwhile In New York City (Caught On Ring Camera) #shorts.mp4")

In [18]:
def main_fight(video_path, accuracyfight=0.65):
  tmp_obj = {}

  if os.path.exists('./tmp.mp4'):
    os.remove('./tmp.mp4')

  # Open the local video file
  cap = cv2.VideoCapture(video_path)

  # Check if the video file was opened successfully
  if not cap.isOpened():
    print("Error: Unable to open video file")
    return None

  else:
    # Open a file to write the video data
    with open("tmp.mp4", "wb") as file:
      while True:
        # Read a frame from the video
        ret, frame = cap.read()
        # Check if the frame was read successfully
        if not ret:
          break  # Break the loop if no more frames are available
        # Write the frame data to the file
        file.write(frame)

  vid = capture("tmp.mp4", 40, 3, 170, 170)
  v = np.array([vid])

  # boom = v.flatten() == 0
  # print('ok: ', np.sum(boom) == len(boom))

  v = torch.from_numpy(v)
  v = v.to(device, dtype=torch.float)
  millis = int(round(time.time() * 1000))
  f, precent = pred_fight(model22, v, acuracy=accuracyfight)
  tmp_obj = {'fight': f, 'precentegeoffight': str(precent)}
  millis2 = int(round(time.time() * 1000))
  tmp_obj['processing_time'] = str((millis2-millis)/1000)
  return tmp_obj

In [19]:
main_fight(
    video_path='./Meanwhile In New York City (Caught On Ring Camera) #shorts.mp4')

Error: Unable to open video file


{'fight': True, 'precentegeoffight': '0.66339463', 'processing_time': '4.446'}